## Research Questions Focus Areas

- Note: Please endeavour to explicitly comment your codes and properly document whichever functions created so as to help other collaborators learn from your codes quicker. Remember that the project is also a learning process.

---

### Dataset Import

In [2]:
import pandas as pd

In [3]:
alltweets = pd.read_csv('citizensvoice_dataset.csv')

In [4]:
alltweets.head(10)

,Unnamed: 0,time_created,tweet,loca_tion
0,0,2022-10-25T23:44:56+00:00,"Tinubu Is An Emperor; Buhari, Osinbajo, Govern...",Nigeria
1,1,2022-10-25T23:37:40+00:00,Dear @PeterObi please stop putting our future ...,"Lagos, Nigeria"
2,2,2022-10-25T23:31:19+00:00,Wike pointed to how the PDP presidential candi...,Nigeria
3,3,2022-10-25T23:03:57+00:00,@fkeyamo @apc_lagos https://t.co/KrKdTG8prX,"Ogun, Nigeria"
4,4,2022-10-27T23:59:39+00:00,"PDP is in total chaos in Ogun, dead in Lagos, ...","Ogun, Nigeria"
5,5,2022-10-20T23:59:41+00:00,One of their lekki lies . They lie like their ...,"Port Harcourt, Nigeria"
6,6,2022-10-25T22:40:58+00:00,@Taikaz_adufe @valeron31 @Cold_n_dark @SugarCh...,"Lagos, Nigeria"
7,7,2022-10-25T22:40:03+00:00,@Taikaz_adufe @valeron31 @Cold_n_dark @SugarCh...,"Lagos, Nigeria"
8,8,2022-10-25T22:31:22+00:00,I just saw video of tinubu threatening to redu...,"Lagos, Nigeria"
9,9,2022-10-20T23:59:34+00:00,IPOB: South East senators beg Buhari to releas...,Nigeria


### General Trends 

- This sections covers the generic trends existing amongst citizens' discussion groups.
        - What is most talked about (regardless of area or topic)

### Citizens' Sentiment

- This section covers citizens' reactions and general sentiment towards certain topic (e.g areas of developments, policies, politically significant events, public office holders' performance and so on).
        - What is the general sentiment of the citizens?
		- What is most discussed(election and governance related)?
		- What is the sentiment towards what is being discussed? 

### Complaint Areas

- This section covers the extraction of various areas of complaints and dissatisfaction amongst citizens (in different aspects of government).
        - What are the various areas of complaints as regards to governance?
		- What are the levels of sentiment towards the various area of complaints?

### Politician's Reputation

- This section covers what citizens's think about certain public office holders, their sentiment towards these individuals and their general popularity or notoriety.
        - Who is most talked about?
		- Popularity or notoreity of the most talked about.
		- Most popular, and most notorious candidates/politician.
		- How much is a certain candidate being talked about?
		- What is being said about each candidate?
		- What is the general sentiment of what is being said?